
# 0) Versioning

#### V10a: First attempt on entire DUS-budget !

#### V04b: further evolution: "Wert" vs. "Ergebnis" now added plus MELT instead of UNSTACK to establish long format

##### V04a: final shape processing (long format) and enrichment before saving the result; tbc if alreay elimination of part-sums or flagging thru data enrichment (e.g. "data point" vs. "data aggregated" or "Wert" vs. "Ergebnis").

##### V03a: Looping table extraction over all pages defined in the control structure, coming up with total data from document >> will require final enrichment (Plan-Info) and transformation (pivot for long format) before saving result

##### V02b:

##### V02a: starting to extract tables from relevant pages based on establishe control structure

##### V01b: starting to establish control structure for table extraction

##### V01a: first version extracting text from file in two different ways (text-extract and parsing)

In [1]:
# prior PDF-treatment:
# (important because different treatments can lead to different control elements in PDF-file
#  to which the code below can react sensibly)
# 1) Download file from DUS-Server
# 2) Eliminate pages with Apple Preview
# 3) Print edited PDF as PDF to Mac
#   >> landscape pages are turned
#   >> transfer to Windows via OneDrive
# 4) Turning landscape pages back to horizontal orientation with XODO
#   >> the parsing reacts to the orientation change, but not the text-extraction !

# Alternative to steps 3) and 4) above:
#   >> turn landscape pages directly in Mac Preview & safe !!
#   >> this leads to the partly different control symbol structure between "Ergebnis" and "Finanzplan"

# 1) Package Import

In [2]:
import pandas as pd

In [3]:
import camelot

In [4]:
import io
from io import StringIO
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
######################################
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser


In [5]:
import re
import xlsxwriter

# 2) Variable declartations (global)

In [6]:
#####################################################################
s_filepath = 'haushaltsplan_2020_mac_pdf_export_landscape.pdf'
####################################################################

In [7]:
l_control_table_cols = ['SeitenNr', 'DezNr', 'DezName', 'ProdNr', 'ProdName', 'PlanTyp']

In [8]:
l_relevante_seiten = ['Teilergebnisplan', 'Teilfinanzplan']

In [9]:
########################################################################################################
# >> must be adjusted for extractions of different periods aka when reading file from different years!!!
########################################################################################################

# needed during the data enrichment process in order to mark the status for a given year
d_year_status = {'2018': 'IST', '2019': 'HH-Ansatz', '2020': 'HH-Ansatz', '2021': 'FinanzP',
                 '2022': 'FinanzP', '2022': 'FinanzP'}

# similar need for adjustement: the dictionary to stick the right year label to columns extracted from tables
d_year_col_rename = {0:'LfdNr', 1:'Position', 2:'2018', 3: '2019', 4:'2020', 5:'2021', 6: '2022', 7: '2023'}

# df-structure of extracted data AFTER extraction and first enrichment
# but BEFORE establishing pivoted & further enriched output dataframe
l_extract_df = ['LfdNr', 'Position', '2018', '2019', '2020', '2021', '2022', '2023', 'DezNr', 'DezName',
                'ProdNr', 'ProdName', 'PlanTyp', 'WertTyp']

#########################################################################################################
# re-arranging and indexing result df for final output:
#########################################################################################################
# list with re-arrange columns to create multi-index
l_col_name_rearrange = ['LfdNr', 'DezNr', 'DezName', 'ProdNr', 'ProdName', 'PlanTyp', 'WertTyp', 'Position',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023']
# list as basis for MELT-operation
l_col_melt_id_vars = ['LfdNr','DezNr','DezName','ProdNr','ProdName','PlanTyp','WertTyp','Position']
l_col_melt_vars_names =  ['2018','2019','2020','2021','2022','2023']

# 3) Definitions

## 3a) Class definitions

## 3b) Function Definitions

In [10]:
def f_extract_text_by_page(pdf_path):
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh, 
                                      caching=True,
                                      check_extractable=True):
            resource_manager = PDFResourceManager()
            fake_file_handle = io.StringIO()
            converter = TextConverter(resource_manager, fake_file_handle)
            page_interpreter = PDFPageInterpreter(resource_manager, converter)
            page_interpreter.process_page(page)
            
            text = fake_file_handle.getvalue()
            yield text
    
            # close open handles
            converter.close()
            fake_file_handle.close()
    

In [11]:
def f_extract_text(pdf_path):
    l_return = []
    for page in f_extract_text_by_page(pdf_path):
        l_return.append(page)
        #print(page)

    return l_return

In [12]:
def f_parse_total_text (pdf_path):
    output_string = StringIO()
    with open(pdf_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

    return output_string.getvalue()

In [13]:
def f_pdf_table_extract (loop_idx, extract_page):

    c_tables = camelot.read_pdf(s_filepath, pages = extract_page)
    df_w1 = c_tables[0].df
    # code to eliminate first row ! But needs to do copy to new df to make it permanent
    df_w2 = df_w1[1:][:].copy()

    # eliminating extra column from "Verpflichtungserklärung"
    v_no_cols = len(list(df_w2.columns.values)) # determine number of columns  (should be 8 aka [0:7])
    if v_no_cols > 8:
        df_w2 = f_col_adj_ermaechtigung(df_w2)

    # renaming columns with dictionary
    df_w2.rename(columns = d_year_col_rename,inplace=True)

    # Making things look nicer:
    # clean '\n' controls from "Position"-column (remains from PDF control codes)
    df_w2['Position'] =  df_w2['Position'].apply(lambda x: re.sub(r'[\n]',' ', str(x)))
    df_w2.head(5)

    # now enriching the table with the referential information from the control data frame !
    df_w2['DezNr'] = df_extract_control['DezNr'][loop_idx]
    df_w2['DezName'] = df_extract_control['DezName'][loop_idx]
    df_w2['ProdNr'] = df_extract_control['ProdNr'][loop_idx]
    df_w2['ProdName'] = df_extract_control['ProdName'][loop_idx]
    df_w2['PlanTyp'] = df_extract_control['PlanTyp'][loop_idx]

    # marking every value either as an atomic value or as a (sub-)total
    # ONLY WORKS if all Positons are either marked with "=" (for calculation results)
    # or '+' and '-' for atomic values
    # this value does not come from the control structure put from the first 
    # character of the "Position"-string
    df_w2['WertTyp'] =  df_w2['Position'].apply(lambda x: 'ERGEBNIS' if x[0] == "=" else 'WERT' )

    return df_w2

In [14]:
def f_col_adj_ermaechtigung (df):
    #####################################################################
    # if other / different columns suddenly show up, this part of code
    # would eventually need to be adjusted; currently, it deleted an
    # additional column after the current year, displaying a "thereof"-info, not the expexted 2021-values
    #####################################################################
    d_drop_col_rename = {5: 'dropcol'} 
    d_col_name_shift = {6: 5, 7: 6, 8: 7}
    
    df.rename(columns = d_drop_col_rename,inplace=True)
    df = df.drop('dropcol', axis=1).copy()
    df.rename(columns = d_col_name_shift,inplace=True)
    
    return df
    

# 4) Actual Program

## 4a) Extract text from file from PDF

In [15]:
# text file is used to establish a control data structure
# that guides the subsequent table extraction

In [16]:
# core page-by-page text extraction extraction !!!!

l_pdf_text_extract_per_page = f_extract_text(s_filepath)

len(l_pdf_text_extract_per_page)

#for i in l_pdf_text_per_page:
#    print (i)

1267

In [17]:
# Alternative approach:
# parsing instead of extracting entire text
# text extraction worked with two-layered approach: 1st loop (function call) calls the entire document,
# the second loop (function call) cycles thru the single pages of the document to that the result is already
# returned on a per-page basis in the result list
# The parsing approach goes the other way around: the entire document with all pages is parsed in one go
# and then the pages seperated via a split of the return string.
 
s_aux = f_parse_total_text(s_filepath)

# because the entire document was parsed, the single long return string needs
# to be split into a list per page; '\X0C' seems to be the PDF control sequence for a page change
l_pdf_text_parse_per_page = s_aux.split("\x0c")

# delete last item from list to make it same length
# last list item is only an empty page, because the split-method encounters a last "\x0c"
del l_pdf_text_parse_per_page[-1]
len(l_pdf_text_parse_per_page)

1267

In [18]:
# result so far: we have two list with each element of the list corresponding to the text of one single
# pdf page of the source pdf file; the difference between the two is that the "text_extract"-list only
# contains the pure text with no control characters whereas the "text_parse"-list has also parsed the
# control strings into the string list-element.
# Funnily enough, the sequence in which the string records the data from the PDF varies slightly between
# the two methods. E.g. "vorläufiges JahresergebnisHaushaltsansatzFinanzplanzeitraum" is in a different 
# position

In [19]:
### Important control code if anything goes wrong wiht the PDF interpretation
i = 134
l_pdf_text_extract_per_page[i]

'TeilfinanzplanDEZ02Dezernat 021111107BgA PersonengesellschaftenJahresergebnisHaushaltsansatzVerpflichtungs-ermächtigungFinanzplanzeitraumNr.BezeichnungInvestitionstätigkeiten2018201920202021202220231+Einzahlungen aus Zuwendungen für Investitionsmaßnahmen0,000000002+Einzahlungen aus der Veräußerung von Sachanlagen0,000000003+Einzahlungen aus der Veräußerung von Finanzanlagen0,000000004+Einzahlungen aus Beiträgen u.ä. Entgelten0,000000005+Sonstige Investitionseinzahlungen0,000000006=Einzahlungen aus Investitionstätigkeit0,000000007-Auszahlungen für den Erwerb von Grundstücken und Gebäuden0,000000008-Auszahlungen für Baumaßnahmen0,000000009-Auszahlungen für den Erwerb von beweglichem Anlagevermögen0,0000000010-Auszahlungen für den Erwerb von Finanzanlagen0,0000000011-Auszahlungen von aktivierbaren Zuwendungen0,0000000012-Sonstige Investitionsauszahlungen0,0000000013=Auszahlungen aus Investitionstätigkeit0,0000000014=Saldo aus Investitionstätigkeit Summe0,00000000\x0c'

In [20]:
l_pdf_text_parse_per_page[i]

'Teilfinanzplan\nDEZ02\n1111107\n\nNr. Bezeichnung\n\nDezernat 02\nBgA Personengesellschaften\n\nInvestitionstätigkeiten\n+ Einzahlungen aus Zuwendungen für Investitionsmaßnahmen\n+ Einzahlungen aus der Veräußerung von Sachanlagen\n+ Einzahlungen aus der Veräußerung von Finanzanlagen\n+ Einzahlungen aus Beiträgen u.ä. Entgelten\n+ Sonstige Investitionseinzahlungen\n= Einzahlungen aus Investitionstätigkeit\n- Auszahlungen für den Erwerb von Grundstücken und Gebäuden\n- Auszahlungen für Baumaßnahmen\n- Auszahlungen für den Erwerb von beweglichem \n\nAnlagevermögen\n\n- Auszahlungen für den Erwerb von Finanzanlagen\n- Auszahlungen von aktivierbaren Zuwendungen\n- Sonstige Investitionsauszahlungen\n= Auszahlungen aus Investitionstätigkeit\n= Saldo aus Investitionstätigkeit Summe\n\n1\n2\n3\n4\n5\n6\n7\n8\n9\n\n10\n11\n12\n13\n14\n\nJahresergebnis\n2018\n\nHaushaltsansatz\n2019\n\n2020\n\nVerpflichtungs-\nermächtigung\n\nFinanzplanzeitraum\n\n2021\n\n2022\n\n2023\n\n0,00\n0,00\n0,00\n0,00\n

In [21]:
# SAME structure
# 'Teilergebnisplan\nDEZ02\n1111107\n\nDezernat 02\nBgA Personengesellschaften\n\nNr. Bezeichnung\n1\
# 'Teilfinanzplan\nDEZ02\n1111107\n\nDezernat 02\nBgA Personengesellschaften\n\nNr. Bezeichnung\n

# DIFFERENT struture
# 'Teilergebnisplan\nDEZ01\n1111101\n\nDezernat 01\nÖffentlichkeitsarbeit\n\nNr. Bezeichnung
# 'Teilfinanzplan\nDEZ01\n1111101\n\nNr. Bezeichnung\n\nDezernat 01\nÖffentlichkeitsarbeit\n


#>>>> adapt code in long loop (if/else-structure)> for i in range(len(l_criteria_ctrl_list))

## 4b) Establish controle structure to manage the table extraction

In [22]:
# get control structure in two different formats, list and dataframe, and initialize
# the two result variables as empty variables
df_extract_control = pd.DataFrame(columns = l_control_table_cols)
l_extract_ctrl = []

In [23]:
df_extract_control.head(3)

,SeitenNr,DezNr,DezName,ProdNr,ProdName,PlanTyp


In [24]:
# extract list 
l_criteria_ctrl_list = []
for element in l_pdf_text_parse_per_page:
    # The first elements contain the "pay load info" of the page which helps to determin 
    # if a page has relevant tables to extract (only "Teilergebnisplan" and "Finanzergebnisplan")
    l_criteria_ctrl_list.append(element.split("\n")[:8])
# print (l_criteria_ctrl_list)

In [25]:
for i in range(len(l_criteria_ctrl_list)):
    l_ctrl = [] # initiate the ctrl list
    # check for relevant page title (see variable definition)
    if l_criteria_ctrl_list[i][0] not in l_relevante_seiten:
        continue

    # check if the 3rd position is numeric; pages for the entire Dezernat do NOT carry
    # a numeric information in 3rd place, as they do not show a product number
    elif l_criteria_ctrl_list[i][2].isnumeric() != True:
        continue

    else:
        # Only relevant pages come to this point here; so now it's time to collect the 
        # "payload info", aka the relevant information to manage and control the table extraction
        # Seiten-Zahl
        l_ctrl.append(str(i+1))
        # Dezernatsnummer
        l_ctrl.append(l_criteria_ctrl_list[i][1])

        # Derzernatsnamen (last PDF used showed slight difference in parsing result of both plans)
        # Differences can be easily identified by looking at the details of l_pdf_text_parse_per_page[i]
        # for several index values
        if l_criteria_ctrl_list[i][0] == 'Teilergebnisplan':
            l_ctrl.append(l_criteria_ctrl_list[i][4])
        else:
            l_ctrl.append(l_criteria_ctrl_list[i][6]) # changed from '6' in earlier PDF
        # Produktnummer
        l_ctrl.append(l_criteria_ctrl_list[i][2])

        # Produktnamen (last PDF used showed slight difference in parsing result of both plans)
        if l_criteria_ctrl_list[i][0] == 'Teilergebnisplan':
            l_ctrl.append(l_criteria_ctrl_list[i][5])
        else:
            l_ctrl.append(l_criteria_ctrl_list[i][7]) # changed from '7' in earlier PDF       
 
        # PlanType
        l_ctrl.append(l_criteria_ctrl_list[i][0])        
        
        
        # Get control info in two different formats:
        # format 1: list
        l_extract_ctrl.append(l_ctrl)    # get control output for all relevant pages as list

        # format 2:...and get control output for all relevant pages also as dataframe
        d_ctrl = dict(zip(l_control_table_cols, l_ctrl))
        # print(d_response)
        df_extract_control = df_extract_control.append(d_ctrl, ignore_index = True)

In [26]:
df_extract_control.head(12)
#l_extract_ctrl

,SeitenNr,DezNr,DezName,ProdNr,ProdName,PlanTyp
0,87,DEZ01,Dezernat 01,1111101,Öffentlichkeitsarbeit,Teilergebnisplan
1,88,DEZ01,Dezernat 01,1111101,Öffentlichkeitsarbeit,Teilfinanzplan
2,94,DEZ01,Dezernat 01,1111114,Betreuung der Bezirksvertretungen,Teilergebnisplan
3,95,DEZ01,Dezernat 01,1111114,Betreuung der Bezirksvertretungen,Teilfinanzplan
4,100,DEZ01,Dezernat 01,1111115,Verwaltungs- und Konzernführung,Teilergebnisplan
5,101,DEZ01,Dezernat 01,1111115,Verwaltungs- und Konzernführung,Teilfinanzplan
6,106,DEZ01,Dezernat 01,1111123,Fraktionen,Teilergebnisplan
7,107,DEZ01,Dezernat 01,1111123,Fraktionen,Teilfinanzplan
8,112,DEZ01,Dezernat 01,5353601,Breitbandversorgung,Teilergebnisplan
9,113,DEZ01,Dezernat 01,5353601,Breitbandversorgung,Teilfinanzplan


In [27]:
len(df_extract_control)

364

In [28]:
len(l_extract_ctrl)

364

## 4c) Actual Table extraction (and refinement) based on control structure

In [29]:
#######################################################
# Loop mit Extraktion der festgestellten relevanten Seiten mittels Camelot-Package
#######################################################

In [30]:
df_out1 = pd.DataFrame(columns = l_extract_df)
df_out1.head(1)

,LfdNr,Position,2018,2019,2020,2021,2022,2023,DezNr,DezName,ProdNr,ProdName,PlanTyp,WertTyp


In [31]:
# core loop to cycle thru all relevant pages stored in the control structure
for p in range(len(df_extract_control)):
    v_page = df_extract_control['SeitenNr'][p]
    df_out1 = pd.concat([df_out1, f_pdf_table_extract(p, v_page)], axis=0) # function needs
    # both the page-no. String AND the idx of the loop to retrieve info from the control strucure

## 4d) Final data processing (long format) and enhancement before outpu

In [32]:
# resetting index
df_out1.reset_index(inplace = True, drop = True)
# and now rearrange the columns in more human-readable sequence
df_out1 = df_out1[l_col_name_rearrange].copy()
df_out1.head(2)

,LfdNr,DezNr,DezName,ProdNr,ProdName,PlanTyp,WertTyp,Position,2018,2019,2020,2021,2022,2023
0,1,DEZ01,Dezernat 01,1111101,Öffentlichkeitsarbeit,Teilergebnisplan,WERT,+ Steuern und ähnliche Abgaben,"0,00",0,0,0,0,0
1,2,DEZ01,Dezernat 01,1111101,Öffentlichkeitsarbeit,Teilergebnisplan,WERT,+ Zuwendungen und allgemeine Umlagen,"3.823,36",3.451,3.800,1.500,300,300


In [33]:
# Establish long format by using MELT on the dataframe
df_out2 = pd.melt(df_out1, id_vars = l_col_melt_id_vars, value_vars = l_col_melt_vars_names, 
        var_name = "Jahr", value_name='EUR')
#df_out2.head(2)

In [34]:
df_out2['PlanVersion'] = 'DUS_HHP_20'

In [35]:
df_out2.head(2)

,LfdNr,DezNr,DezName,ProdNr,ProdName,PlanTyp,WertTyp,Position,Jahr,EUR,PlanVersion
0,1,DEZ01,Dezernat 01,1111101,Öffentlichkeitsarbeit,Teilergebnisplan,WERT,+ Steuern und ähnliche Abgaben,2018,"0,00",DUS_HHP_20
1,2,DEZ01,Dezernat 01,1111101,Öffentlichkeitsarbeit,Teilergebnisplan,WERT,+ Zuwendungen und allgemeine Umlagen,2018,"3.823,36",DUS_HHP_20


In [36]:
writer = pd.ExcelWriter('_HH_DUS_vollstaendig_test.xlsx', engine='xlsxwriter')
df_out1.to_excel(writer, sheet_name='wide_data', engine='xlsxwriter')
df_out2.to_excel(writer, sheet_name='long_data', engine='xlsxwriter')
writer.save()